In [5]:
print('ok')

ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import CTransformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as LangchainPinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

c:\Users\mouny\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import os
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECODE_API_KEY")

In [23]:
print(PINECONE_API_KEY)

pcsk_22zsdy_GNv7rp5GdUYya9XFAzN5G3grStKB2zkKthzKJ3dKEsHRJVKvNoEFNJK9dgAUJ3T


In [26]:
import os
PINECONE_API_KEY='pcsk_22zsdy_GNv7rp5GdUYya9XFAzN5G3grStKB2zkKthzKJ3dKEsHRJVKvNoEFNJK9dgAUJ3T'
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY



In [24]:
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
   
    return documents 

In [25]:
extracted_data=load_pdf("data/")

In [27]:
#Create text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    docs = text_splitter.split_documents(extracted_data)
    return docs

In [28]:
text_chunks=text_split(extracted_data)
print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [29]:
from sentence_transformers import SentenceTransformer
def download_hugging_face_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [30]:
embeddings=download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [31]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [32]:
from pinecone import Pinecone as PineconeClient, ServerlessSpec
pc=PineconeClient(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"





In [33]:
# store texts in Pinecone via LangChain
texts = [t.page_content for t in text_chunks]
docsearch = LangchainPinecone.from_texts(texts, embedding=embeddings, index_name=index_name)

In [34]:

query="What are Allergies?"
docs=docsearch.similarity_search(query, k=3)
print('Result', docs)

Result [Document(metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the allergen and whether the allergen spread sys-'), Document(metadata={}, page_content='reactions is triggered by harmless, everyday substances.\nThis is the condition known as allergy, and the offend-\ning substance is called an allergen. Common inhaled\nallergens include pollen, dust, and insect parts from tiny\nhouse mites. Common food allergens include nuts, fish,\nand milk.\nAllergic reactions involve a special set of cells in\nthe immune system known as mast cells. Mast cells\nserve as guards in the tissues where the body meets 

In [35]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [36]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [37]:
import os
print(os.path.exists(r"C:\Users\mouny\medical-chatbot\model\llama-2-7b-chat.Q4_0.gguf"))


True


In [17]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained(
    r"C:\Users\mouny\medical-chatbot\model",
    model_file="llama-2-7b-chat.Q4_0.gguf",
    model_type="llama",
    gpu_layers=0
)



In [38]:
# Load the model through LangChain wrapper
llm = CTransformers(
    model=r"C:\Users\mouny\medical-chatbot\model\llama-2-7b-chat.Q4_0.gguf",
    model_type="llama",
    config={
        'gpu_layers': 0,
        'max_new_tokens': 512,
        'temperature': 0.7,
        'context_length': 2048
    }
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)


In [39]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\mouny\AppData\Local\Temp\ipykernel_34364\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :   Acne is a common skin disease characterized by pimples on the face, chest, and back, caused when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :   What is the disease that AIDS is often used as a synonym for?


KeyboardInterrupt: 